![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2019/master/logo.png)

![Plan zajęć](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2019/master/plan_zajec.jpg)

# SELECT Python
## Asyncio
### Wojciech Łuszczyński
#### Python level UP 15.04.2019

# Gdzie na mapie leży AsyncIO

## Znane podejścia:
- Concurrency (Threading) - równoległe
- Parallelism (Multiprocessing) - współbieżne i równoległe

> Współbieżność i równoległość to obszerne tematy, które nie są łatwe do pojęcia.

## Nowy gracz w polu
- Asynchronus (Asyncio) - daje wrażenie współbieżności ...

## AsyncIO
- Jednoprocesowe
- Jednowątkowe
- Współbieżność osiągnięta dzięki korutynom (corutine) w mechaniźmie pętli zdarzeń.
- Asyncio to styl programowania współbieżnego, ale nie jest to równoległość. 

# Programowanie asyncio - po co to komu?

## Pytanie: Co jeszcze da się zoptymalizować w pythonie ?

Trzeba sobie zdać sprawę z tego, że procesory są bardzo szybkie.
Dane wewnątrz struktur procesora przetwarzają się o rząd wielkości szybciej niż dostęp do pamięci ram.
Dostęp do świata zewnętrznego, dyski (nawet ssd), sieć, to są kolejne rzędy wielkości wolniej.
W związku z tym, w programach, które komunikują się z siecią, dyskiem, bazą danych itp, procesor głównie czeka (idle time). 

## Jak odzyskać stracony czas

Stracony czas (oczekiwanie na synchronizację danych) można odzyskać. Możemy użyć "klasycznych" rozwiązań, czyli wielowątkowości lub wieloprocesowosći. Jednak te technologie wiążą się ze stosunkowo dużym kosztem przełączania procesów (switching context). Problem ten wzrasta wraz z obciążeniem systemu. Przy dużej ilości procesów, zaczyna być kosztowny, a także może stanowić główny czas pracy programu. 

Rozwiązaniem problemu przełączania jest eliminacja przełączania procesów. Ideą programowania asynchronicznego jest cykliczne wywoływanie procedur w pętli zdarzeń. W jednym procesie, bez narzutu związanego z przełączaniem procesów. (zastępując go narzutem pętli zdarzeń)

Asynchroniczność jest wspierana bezpośrednio przez kernel linuxowy. AIO (Asynchronous I/O) jest to api pozwalające na operacje wyjścia/wejścia bez blokowania na czas oczekiwania na dane. Zamiast tego, proces może kontynuować pracę, i sprawdzić za jakiś czas czy są dostępne wyniki operacji. 

Link dla zainteresowanych:
- https://oxnz.github.io/2016/10/13/linux-aio/
- https://developer.ibm.com/articles/l-async/

# Jak to robi Python

## Asyncio

Moduł asyncio jest implementacją asynchronicznych operacji wyjścia/wejścia. Oryginalnie moduł ten był zaimplementowany poprzez wykorzystanie generatorów (`yield` i `yield from`). Obecnie (od Python 3.5+) są dedykowane słowa kluczowe takie jak: `await` i `async`. Dodatkowo składnia i boilerplate aplikacji zmniejszył się drastycznie wraz z wejściem Python 3.7+

# UWAGA

- W internecie można znaleźć wiele artykułów, tutoriali i szkoleń dotyczących `asyncio` w Python.
- Zawsze patrzcie dla której wersji Python powstał artykuł !
- Zachęcamy do korzystania z Python w wersji 3.7+ jako że starsze słowa kluczowe (np. `yield from`) są zaplanowane do usunięcia w wesji Python 3.10

## W pętli zdarzeń

EventLoop jest podstawowym konceptem asynchronicznego programowania. Jest to pętla zdarzeń. Czyli cyklicznie wywoływanych `korutyn`.

## Przykład z życia

Mistrz szachowy Judit Polgár prowadzi partię szachów, w której gra wielu graczy. Ma dwa sposoby prowadzenia rozgrywki: synchronicznie i asynchronicznie.


- Założenia:
    - 24 przeciwników
    - Każdy ruch Judit trwa 5 sekund
    - każdy ruch przeciwnika trwa 55 sekund
    - Aby rozegrać partię potrzeba średnio 30 par ruchów (60 ruchów łącznie)


- Wersja synchroniczna:
    - Judit gra jedną grę na raz, nigdy dwie w tym samym czasie, dopóki gra się nie zakończy.
    - każda gra trwa (55 + 5) * 30 == 1800 sekund lub 30 minut.
    - Cała wystawa trwa 24 * 30 == 720 minut lub 12 godzin.


- Wersja asynchroniczna:
     - Judit przesuwa się od stołu do stołu, wykonując jeden ruch przy każdym stole.
     - Opuszcza stół i pozwala przeciwnikowi wykonać następny ruch w czasie oczekiwania.
     - Jeden ruch we wszystkich 24 meczach wylicza się na 24 * 5 == 120 sekund lub 2 minuty.
     - Cała partia jest teraz zredukowana do 120 * 30 == 3600 sekund, czyli tylko 1 godzinę.

## Przykład z kodu

Zróbmy AB test pobierania danych z serwera http. Użyjemy strony http://httpbin.org. Dla pełnego efektu potrzebujemy odpowiedzi serwera, która jest stosunkowo długa. http://httpbin.org/delay/1 generuje odpowiedź serwera która jest generowana przez około 1s 

In [3]:
import requests
import time

def make_sync_requests(max, url):
    start_time = time.time()

    response = []
    for i in range(0, max):
        requests.get(url)
        response.append(i)

    print(response)
    execution_time = time.time() - start_time
    print('execution time: {}'.format(execution_time))

make_sync_requests(10, "http://httpbin.org/delay/1")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
execution time: 12.239115715026855


## Asynchroniczny request

```python
import asyncio
import time
from aiohttp import ClientSession

async def make_asyncio_request(url, index, session):
    await session.get(url)
    return index

async def make_asyncio_requests(n, url):
    start_time = time.time()
    async with ClientSession() as session:
        corutines = [make_asyncio_request(url, index, session) for index in range(0, n)]
        responses = await asyncio.gather(*corutines)
    print(responses)
    execution_time = time.time() - start_time
    print('execution time: {}'.format(execution_time))

async def main():
    await make_asyncio_requests(10, "http://httpbin.org/delay/1")

asyncio.run(main())
```

### Output
```python
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
execution time: 2.561192035675049s
[Finished in 2.7s]
```

# Od czego zacząć przygodę z asyncio

## Jak uruchomić korutynę

```python
import asyncio

async def main():
    await asyncio.sleep(10)

asyncio.run(main())
```

## Asynchroniczne Hello World aka Uruchomienie wielu korutyn i czekanie na ich rezultaty

```python
import asyncio
import time

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

s = time.time()
asyncio.run(main())
t = time.time() - s
print("time spent: {:0.2f} seconds.".format(t))
```

**Results:**
```python
One
One
One
Two
Two
Two
time spent: 1.00 seconds.
[Finished in 1.1s]
```

## Uruchomienie wielu korutyn

```python
import time, asyncio
start = time.time()

def time_log():
    return 'at {:1.4f} seconds'.format((time.time() - start))

async def cor1():
    print('cor1 started work: {}'.format(time_log()))
    await asyncio.sleep(2)
    print('cor1 finished work: {}'.format(time_log()))

async def cor2():
    print('cor2 started work: {}'.format(time_log()))
    await asyncio.sleep(2)
    print('cor2 finished work: {}'.format(time_log()))

async def cor3():
    print("Let's work while the coroutines are blocked, {}".format(time_log()))
    await asyncio.sleep(1)
    print("Done!")

async def main():
    tasks = [cor1(), cor2(), cor3()]
    await asyncio.gather(*tasks)

asyncio.run(main())
```

**Results:**
```python
gr1 started work: at 0.0 seconds
gr2 started work: at 0.0 seconds
Let's do some stuff while the coroutines are blocked, at 0.0 seconds
Done!
gr1 ended work: at 2.0 seconds
gr2 Ended work: at 2.0 seconds
```

## Uruchomienie wielu korutyn i wykorzystanie ich rezultatów kiedy to tylko możliwe


```python
import asyncio

async def important_task(index):
    print('important_task start executing')
    await asyncio.sleep(index)
    return index

async def main():
    corutines = [important_task(index) for index in range(10, 0, -1)]
    for i, future in enumerate(asyncio.as_completed(corutines)):
        result = await future
        print('{} {}'.format(">>" * (i + 1), result))

asyncio.run(main())
```

**Results:**
```python
important_task start executing
.... x10
>> 1
>>>> 2
>>>>>> 3
>>>>>>>> 4
>>>>>>>>>> 5
>>>>>>>>>>>> 6
>>>>>>>>>>>>>> 7
>>>>>>>>>>>>>>>> 8
>>>>>>>>>>>>>>>>>> 9
>>>>>>>>>>>>>>>>>>>> 10
```

## as_completed() vs gather()

- `gather()` rozpoczyna wszyskie corutyny w kolejności ich zgłoszenia i zwraca wyniki w tej samej kolejności.
- `gather()` defacto czeka na wykonanie wszystkich korutyn by zwrócić poprawny wynik
- `as_completed()` rozpoczyna wszystkie corutyny "jak leci"
- `as_completed()` zwraca generator którego zwracane wartości to kolejne wykonane korutyny, nie trzyma kolejności.

## Łączenie korutyn w łańcuchy (Chaining Coroutines)

```python
import asyncio
import random
import time


async def randint(a, b):
    return random.randint(a, b)


async def part1(n):
    i = await randint(0, 10)
    print(f"part1({n}) sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = "result{}-1".format(n)
    print("Returning part1({}) == {}.".format(n, result))
    return result


async def part2(n, arg):
    i = await randint(0, 10)
    print(f"part2{n, arg} sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = "result{}-2 derived from {}".format(n, arg)
    print("Returning part2({},{}) == {}.".format(n, arg, result))
    return result


async def chain(n):
    start = time.time()
    p1 = await part1(n)
    p2 = await part2(n, p1)
    print("-->Chained result{} => {} (took {:0.2f} seconds).".format(
        n, p2, time.time() - start))


async def main():
    await asyncio.gather(*(chain(n) for n in (1, 2, 3)))

s = time.time()
asyncio.run(main())
t = time.time() - s
print("time spent: {:0.2f} seconds.".format(t))
```

## Mój task jest długi, i blokuje przełączanie pętli

Rozwiązaniem jest podział taska na mniejsze kawałki, które będziemy wyowłaywać przez await lub jako osobne taski.
Mozna także użyć sztuczki: `await asyncio.sleep(0)`

```python
async def long_task():
    for strategy in strategies:
        strategy.make_decision()
        await.sleep(0)       
```

- Nie jest dobrym pomysłem tworzenie korutyn które są czasochłonne z punktu widzenia operacji na CPU.
- Taski absorbujące czas wstrzymują całą pętlę do końca ich wykonania kiedy inne korutyny nawet zakończą swoją pracę IO
- Jednym z rozwiązań jest delegowanie takich tasków do osobnych wątków (SIC!!)

## Gdzie używać `asyncio`

- network io, bez znaczenia czy twój program jest serwerem czy klientem
- serverless, takich jak sieć typu peer-to-peer, wielu użytkowników, takich jak grupowy czat
- jeśli zastanawiamy się nad wielowątkowym programem który wymać będzie bardzo wielu wątków -> użyj `asyncio`
    - właściwie nad każdym wielowątkowym programem, debugowanie Race Conditions i problemu dostępu do pamięci nie jest tego warte
- wszelkie operacje odczytu / zapisu
    - zwłaszcza te w których chcesz naśladować styl „wyślij i zapomnij”, a mniej martwić się o blokowanie tego, z czego czytasz czy do czego piszesz


## Gdzie na pewno nie używać `asyncio`

- Wszystkie jednorodne operacje zachłanne w moc procesora np: mnożenie macierzy, przeszukiwanie wielkich zbiorów danych
- Wszędzie tam gdzie pod spodem wykorzystywane są synchroniczne biblioteki
    - wiele sterowników do baz danych nie ma odpowiedników asynchronicznych
- Jeśli Twoim jedynym pomysłem na asynchroniczność jest wstawianie wszędzie `await` i `async` na siłę. To może w rezultacie spowolnić wykonanie kodu
- *EXPERT STUFF* Paradoksalenie nie używać AsyncIO na serwerach które posiadają bardzo dużo rdzeni procesora a nasz program ma ograniczoną ilość RAMu którą może zająć.
   

# Co dalej?


## Przydatne linki
1. https://docs.python.org/3/library/asyncio.html
1. https://hackernoon.com/asyncio-for-the-working-python-developer-5c468e6e2e8e
1. https://medium.com/python-pandemonium/asyncio-coroutine-patterns-beyond-await-a6121486656f
1. https://medium.com/@yeraydiazdiaz/asyncio-coroutine-patterns-errors-and-cancellation-3bb422e961ff
1. https://magic.io/blog/asyncpg-1m-rows-from-postgres-to-python/
1. https://magic.io/blog/uvloop-blazing-fast-python-networking/
1. https://github.com/channelcat/sanic
1. http://lucumr.pocoo.org/2016/10/30/i-dont-understand-asyncio/

## Włącz tryb debug

asyncio domyślnie pracuje w trybie produkcyjnym, by włączyć tryb debug można:
  - Możesz ustawić zmienną środowiskową `PYTHONASYNCIODEBUG=1`
  - Uruchmoć skrypt pythonowy z parametrem `-X`
  - Przekazać `debug=True` w wyołaniu `asyncio.run(debug=True)`
  - Dodać w kodzie wywołanie `loop.set_debug()`

## Co zyskujesz włączając debug

- asyncio sprawdza które `coroutines` zostały zadeklarowane ale nigdy nie użyte, loguje takie przypadki.
- zamiast wisieć w nieskończoność wiele API asyncio zwróci błąd (np `loop.call_soon()` czy `loop.call_at()`)
- zalogowany zostanie czas połączeń I/O które trwają zdecydowanie za długo
- domyślnie zalogowane zostaną też wszystkie callbacki które zajęły więcej niż 100ms

Dzięki temu ebugowanie aplikacji staje się zancznie łatwiejsze, asyncio samo podpowiada nam które elementy kodu są potrzebne do przerobienia i zoptymalizowania by cała pętla działała jak najszybciej się da.

## Asyncio ma przyzwoity domyślny logger

Należy tylko pamiętać o ustawieniu poziomu debug.

Przykładowa prosta konfiguracja

```python
import logging
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
chandler = logging.StreamHandler()
chandler.setFormatter(formatter)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(chandler)
```